# DATASOURCE

In [27]:
from ruamel import yaml

import great_expectations as gx
from great_expectations.core.batch import Batch, BatchRequest, RuntimeBatchRequest

context = gx.get_context()

In [28]:
datasource_config = {
    "name": "my_s3_datasource",
    "class_name": "Datasource",
    "execution_engine": {"class_name": "PandasExecutionEngine"},
    "data_connectors": {
        "default_runtime_data_connector_name": {
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": ["default_identifier_name"],
        },
        "default_inferred_data_connector_name": {
            "class_name": "InferredAssetS3DataConnector",
            "bucket": "sevir-bucket-01",
            "prefix": "database-files/",
            "default_regex": {
                "pattern": "(.*)\\.csv",
                "group_names": ["data_asset_name"],
            },
        },
    },
}

In [29]:
context.test_yaml_config(yaml.dump(datasource_config))

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_inferred_data_connector_name : InferredAssetS3DataConnector

	Available data_asset_names (2 of 2):
		database-files/goes18_data (1 of 1): ['database-files/goes18_data.csv']
		database-files/nexrad_data (1 of 1): ['database-files/nexrad_data.csv']

	Unmatched data_references (1 of 1):['database-files/sql_scraped_database.db']

	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



In [30]:
context.add_datasource(**datasource_config)

## GOES18 S3

In [5]:
batch_request = RuntimeBatchRequest(
    datasource_name="my_s3_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="goes18_data_asset",  # this can be anything that identifies this data_asset for you
    runtime_parameters={"path": "s3://sevir-bucket-01/database-files/goes18_data.csv"},  # Add your S3 path here.
    batch_identifiers={"default_identifier_name": "default_identifier"},
)

In [6]:
context.add_or_update_expectation_suite(expectation_suite_name="goes18_suite")
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="goes18_suite"
)
print(validator.head())

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

   id       product  year  day  hour
0   1  ABI-L1b-RadC  2022  209     0
1   2  ABI-L1b-RadC  2022  209     1
2   3  ABI-L1b-RadC  2022  209     2
3   4  ABI-L1b-RadC  2022  209     3
4   5  ABI-L1b-RadC  2022  209     4


## Expectations

### Table Expectations

In [7]:
validator.expect_table_row_count_to_be_between(max_value=10000, min_value=0)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": 4912
  }
}

In [8]:
validator.expect_table_columns_to_match_set(column_set=['day', 'year', 'product', 'hour', 'id'])

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": [
      "id",
      "product",
      "year",
      "day",
      "hour"
    ]
  }
}

### Column Expectations

#### ID

In [9]:
validator.expect_column_values_to_be_unique(column='id', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'id'}, 'metric_name': 'column_values.unique.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

In [10]:
validator.expect_column_values_to_not_be_null(column='id', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'id'}, 'metric_name': 'column_values.nonnull.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  }
}

In [11]:
validator.expect_column_values_to_be_between(column='id', max_value=10000, min_value=0, mostly=1.0, strict_max=False, strict_min=False, meta={'profiler_details': {'column_max_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'id'}, 'metric_name': 'column.max', 'metric_value_kwargs': None}, 'num_batches': 1}, 'column_min_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'id'}, 'metric_name': 'column.min', 'metric_value_kwargs': None}, 'num_batches': 1}}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

#### Product

In [12]:
validator.expect_column_values_to_not_be_null(column='product', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'product'}, 'metric_name': 'column_values.nonnull.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  }
}

In [13]:
validator.expect_column_values_to_be_in_set(column='product', mostly=1.0, value_set=['ABI-L1b-RadC'], meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'product'}, 'metric_name': 'column.distinct_values', 'metric_value_kwargs': None}, 'num_batches': 1, 'parse_strings_as_datetimes': False}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

#### Year

In [14]:
validator.expect_column_values_to_not_be_null(column='year', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'year'}, 'metric_name': 'column_values.nonnull.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  }
}

In [15]:
validator.expect_column_values_to_be_between(column='year', max_value=2023, min_value=2022, mostly=1.0, strict_max=False, strict_min=False, meta={'profiler_details': {'column_max_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'year'}, 'metric_name': 'column.max', 'metric_value_kwargs': None}, 'num_batches': 1}, 'column_min_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'year'}, 'metric_name': 'column.min', 'metric_value_kwargs': None}, 'num_batches': 1}}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

#### Day

In [16]:
validator.expect_column_values_to_not_be_null(column='day', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'day'}, 'metric_name': 'column_values.nonnull.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  }
}

In [17]:
validator.expect_column_values_to_be_between(column='day', max_value=365, min_value=1, mostly=1.0, strict_max=False, strict_min=False, meta={'profiler_details': {'column_max_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'day'}, 'metric_name': 'column.max', 'metric_value_kwargs': None}, 'num_batches': 1}, 'column_min_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'day'}, 'metric_name': 'column.min', 'metric_value_kwargs': None}, 'num_batches': 1}}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

#### Hours

In [18]:
validator.expect_column_values_to_not_be_null(column='hour', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'hour'}, 'metric_name': 'column_values.nonnull.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  }
}

In [19]:
validator.expect_column_values_to_be_between(column='hour', max_value=23, min_value=0, mostly=1.0, strict_max=False, strict_min=False, meta={'profiler_details': {'column_max_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'hour'}, 'metric_name': 'column.max', 'metric_value_kwargs': None}, 'num_batches': 1}, 'column_min_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'hour'}, 'metric_name': 'column.min', 'metric_value_kwargs': None}, 'num_batches': 1}}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 4912,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

In [20]:
validator.save_expectation_suite(discard_failed_expectations=False)

In [21]:
my_checkpoint_name = "goes18_checkpoint_v0.1"
checkpoint_config = {
    "name": my_checkpoint_name,
    "config_version": 1.0,
    "class_name": "SimpleCheckpoint",
    "run_name_template": "%Y%m%d-%H%M%S-my-run-name-template",
}

In [22]:
my_checkpoint = context.test_yaml_config(yaml.dump(checkpoint_config))

Attempting to instantiate class from config...
	Instantiating as a SimpleCheckpoint, since class_name is SimpleCheckpoint
	Successfully instantiated SimpleCheckpoint


Checkpoint class name: SimpleCheckpoint
Your current Checkpoint configuration has an empty or missing "validations" attribute.  This
means you must either update your Checkpoint configuration or provide an appropriate validations
list programmatically (i.e., when your Checkpoint is run).
                    


In [23]:
context.add_or_update_checkpoint(**checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "goes18_checkpoint_v0.1",
  "profilers": [],
  "run_name_template": "%Y%m%d-%H%M%S-my-run-name-template",
  "runtime_configuration": {},
  "validations": []
}

In [25]:
checkpoint_result = context.run_checkpoint(
    checkpoint_name=my_checkpoint_name,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": "goes18_suite",
        }
    ],
)

Calculating Metrics:   0%|          | 0/58 [00:00<?, ?it/s]

In [26]:
context.open_data_docs()

## NEXRAD S3

In [31]:
batch_request = RuntimeBatchRequest(
    datasource_name="my_s3_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="nexrad_data_asset",  # this can be anything that identifies this data_asset for you
    runtime_parameters={"path": "s3://sevir-bucket-01/database-files/nexrad_data.csv"},  # Add your S3 path here.
    batch_identifiers={"default_identifier_name": "default_identifier"},
)

In [32]:
context.add_or_update_expectation_suite(expectation_suite_name="nexrad_suite")
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="nexrad_suite"
)
print(validator.head())

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

   id  year  month  day ground_station
0   1  2022      1    1           FOP1
1   2  2022      1    1           KABR
2   3  2022      1    1           KABX
3   4  2022      1    1           KAKQ
4   5  2022      1    1           KAMA


## Expectations

### Table Expectations

In [33]:
validator.expect_table_row_count_to_be_between(max_value=120000, min_value=0)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": 84163
  }
}

In [51]:
validator.expect_table_columns_to_match_set(column_set=['year', 'day', 'month', 'ground_station', 'id'])

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": [
      "id",
      "year",
      "month",
      "day",
      "ground_station"
    ]
  }
}

### Column Expectations

#### ID

In [52]:
validator.expect_column_values_to_be_unique(column='id', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'id'}, 'metric_name': 'column_values.unique.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 84163,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

In [53]:
validator.expect_column_values_to_not_be_null(column='id', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'id'}, 'metric_name': 'column_values.nonnull.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 84163,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  }
}

In [54]:
validator.expect_column_values_to_be_between(column='id', max_value=120000, min_value=0, mostly=1.0, strict_max=False, strict_min=False, meta={'profiler_details': {'column_max_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'id'}, 'metric_name': 'column.max', 'metric_value_kwargs': None}, 'num_batches': 1}, 'column_min_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'id'}, 'metric_name': 'column.min', 'metric_value_kwargs': None}, 'num_batches': 1}}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 84163,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

#### Year

In [55]:
validator.expect_column_values_to_not_be_null(column='year', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'year'}, 'metric_name': 'column_values.nonnull.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 84163,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  }
}

In [56]:
validator.expect_column_values_to_be_between(column='year', max_value=2023, min_value=2022, mostly=1.0, strict_max=False, strict_min=False, meta={'profiler_details': {'column_max_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'year'}, 'metric_name': 'column.max', 'metric_value_kwargs': None}, 'num_batches': 1}, 'column_min_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'year'}, 'metric_name': 'column.min', 'metric_value_kwargs': None}, 'num_batches': 1}}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 84163,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

#### Month

In [57]:
validator.expect_column_values_to_be_between(column='month', max_value=12, min_value=1, mostly=1.0, strict_max=False, strict_min=False, meta={'profiler_details': {'column_max_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'month'}, 'metric_name': 'column.max', 'metric_value_kwargs': None}, 'num_batches': 1}, 'column_min_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'month'}, 'metric_name': 'column.min', 'metric_value_kwargs': None}, 'num_batches': 1}}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 84163,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

In [58]:
validator.expect_column_values_to_not_be_null(column='month', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'month'}, 'metric_name': 'column_values.nonnull.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 84163,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  }
}

#### Day

In [59]:
validator.expect_column_values_to_not_be_null(column='day', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'day'}, 'metric_name': 'column_values.nonnull.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 84163,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  }
}

In [60]:
validator.expect_column_values_to_be_between(column='day', max_value=31, min_value=1, mostly=1.0, strict_max=False, strict_min=False, meta={'profiler_details': {'column_max_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'day'}, 'metric_name': 'column.max', 'metric_value_kwargs': None}, 'num_batches': 1}, 'column_min_values_range_estimator': {'metric_configuration': {'domain_kwargs': {'column': 'day'}, 'metric_name': 'column.min', 'metric_value_kwargs': None}, 'num_batches': 1}}})

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 84163,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

#### Ground Station

In [61]:
validator.expect_column_values_to_not_be_null(column='ground_station', meta={'profiler_details': {'metric_configuration': {'domain_kwargs': {'column': 'ground_station'}, 'metric_name': 'column_values.nonnull.unexpected_count', 'metric_value_kwargs': None}, 'num_batches': 1}})

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 84163,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  }
}

In [62]:
validator.save_expectation_suite(discard_failed_expectations=False)

In [63]:
my_checkpoint_name = "nexrad_checkpoint_v0.2"
checkpoint_config = {
    "name": my_checkpoint_name,
    "config_version": 1.0,
    "class_name": "SimpleCheckpoint",
    "run_name_template": "%Y%m%d-%H%M%S-my-run-name-template",
}

In [64]:
my_checkpoint = context.test_yaml_config(yaml.dump(checkpoint_config))

Attempting to instantiate class from config...
	Instantiating as a SimpleCheckpoint, since class_name is SimpleCheckpoint
	Successfully instantiated SimpleCheckpoint


Checkpoint class name: SimpleCheckpoint
Your current Checkpoint configuration has an empty or missing "validations" attribute.  This
means you must either update your Checkpoint configuration or provide an appropriate validations
list programmatically (i.e., when your Checkpoint is run).
                    


In [65]:
context.add_or_update_checkpoint(**checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "nexrad_checkpoint_v0.2",
  "profilers": [],
  "run_name_template": "%Y%m%d-%H%M%S-my-run-name-template",
  "runtime_configuration": {},
  "validations": []
}

In [66]:
checkpoint_result = context.run_checkpoint(
    checkpoint_name=my_checkpoint_name,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": "nexrad_suite",
        }
    ],
)

Calculating Metrics:   0%|          | 0/53 [00:00<?, ?it/s]

In [67]:
context.open_data_docs()